## Exercise 1

In [1]:
from skimage import color, io, measure, img_as_ubyte, exposure
from skimage.measure import profile_line
from skimage.transform import rescale, resize
import matplotlib.pyplot as plt
import numpy as np
import pydicom as dicom

ImportError: /home/chris/anaconda3/envs/course02502/lib/python3.9/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /home/chris/anaconda3/envs/course02502/lib/python3.9/site-packages/scipy/fft/_pocketfft/pypocketfft.cpython-39-x86_64-linux-gnu.so)

In [2]:
# Directory containing data and images
in_dir = "data/"

# X-ray image
im_name = "vertebra.png"

# Read the image.
# Here the directory and the image name is concatenated
# by "+" to give the full path to the image.
im_org = io.imread(in_dir + im_name)

In [3]:
io.imshow(im_org)
plt.title('Vertebra image')
io.show()

## Exercise 2

In [4]:
np.min(im_org)

In [5]:
np.max(im_org)

In [6]:
io.imshow(im_org, vmin=57, vmax=235)
plt.title('Vertebra image (with gray level scaling)')
io.show()

## Exercise 3

In [7]:
from skimage.util import img_as_float
from skimage.util import img_as_ubyte

In [8]:
float_img = img_as_float(im_org)

In [9]:
np.min(float_img)

In [10]:
np.max(float_img)

## Exercise 4

In [11]:
ubyte_img = img_as_ubyte(float_img)

In [12]:
np.min(ubyte_img)

In [13]:
np.max(ubyte_img)

## Exercise 5

In [14]:
def histogram_stretch(img_in):
    """
    Stretches the histogram of an image 
    :param img_in: Input image
    :return: Image, where the histogram is stretched so the min values is 0 and the maximum value 255
    """
    # img_as_float will divide all pixel values with 255.0
    img_float = img_as_float(img_in)
    min_val = img_float.min()
    max_val = img_float.max()
    min_desired = 0.0
    max_desired = 1.0
	
    img_out = (max_desired - min_desired) / (max_val - min_val) * (img_float - min_val) + min_desired

    # img_as_ubyte will multiply all pixel values with 255.0 before converting to unsigned byte
    return img_as_ubyte(img_out)

## Exercise 6

In [15]:
stretched_img = histogram_stretch(im_org)
io.imshow(stretched_img)
plt.title('Vertebra image after hist stretching')
io.show()

## Exercise 7

In [16]:
def gamma_map(img, gamma):
    float_img = img_as_float(img)
    gamma_map = np.power(float_img, gamma)
    
    return gamma_map

## Exercise 8

In [17]:
gamma_mapped_img = gamma_map(im_org, 4)
io.imshow(gamma_mapped_img)
plt.title('Vertebra image after hist stretching')
io.show()

## Exercise 9

In [18]:
def threshold_image(img_in, thres):
    """
    Apply a threshold in an image and return the resulting image
    :param img_in: Input image
    :param thres: The treshold value in the range [0, 255]
    :return: Resulting image (unsigned byte) where background is 0 and foreground is 255
    """
    mask = img_in > thres
    return img_as_ubyte(mask)

## Exercise 10

In [ ]:
im_thres = threshold_image(im_org, thres = 115)

fig, ax = plt.subplots(nrows=1, ncols = 3, figsize = (12, 5))
ax[0].imshow(im_org, cmap = 'gray', vmin = 0, vmax = 255)
ax[0].set_title('Original image')
ax[1].imshow(im_thres, cmap = 'gray', vmin = 0, vmax = 255)
ax[1].set_title('Foreground image')
ax[2].imshow(im_org, cmap = 'gray', vmin = 0, vmax = 255)
ax[2].contour(im_thres, [254, 256])
ax[2].set_title('Mask contour')
plt.show()

In [ ]:
from skimage.filters import threshold_otsu

## Exercise 11

In [ ]:
otsu_threshold = threshold_otsu(im_org)
otsu_img = threshold_image(im_org, otsu_threshold)

fig, ax = plt.subplots(nrows=1, ncols = 2, figsize = (9, 5))
ax[0].imshow(im_thres, cmap = 'gray', vmin = 0, vmax = 255)
ax[0].set_title('Manual threshold image')
ax[1].imshow(otsu_img, cmap = 'gray', vmin = 0, vmax = 255)
ax[1].set_title('Otsu threshold image')
plt.show()

## Exercise 12

In [38]:
im_org = io.imread(in_dir + 'dark_background.png')
from skimage.color import rgb2gray

gray_img = rgb2gray(im_org)
gray_img = img_as_ubyte(gray_img)

otsu_img = threshold_image(gray_img, threshold_otsu(gray_img))
manual_thres_img = threshold_image(gray_img, 5)

fig, ax = plt.subplots(nrows=1, ncols = 3, figsize = (9, 5))

ax[0].imshow(gray_img, cmap = 'gray', vmin = 0, vmax = 255)
ax[0].set_title('Gray Image')
ax[0].axis('off')
ax[1].imshow(otsu_img, cmap = 'gray', vmin = 0, vmax = 255)
ax[1].set_title('Otsu threshold')
ax[1].axis('off')
ax[2].imshow(manual_thres_img, cmap = 'gray', vmin = 0, vmax = 255)
ax[2].set_title('Otsu threshold')
ax[2].axis('off')
plt.show()

## Exercise 13

In [ ]:
im_org = io.imread(in_dir + 'DTUSigns2.jpg')
io.imshow(im_org)
plt.title('DTU sign image')
io.show()

In [33]:
def detect_dtu_signs(img_in, sign):
    r_comp = im_org[:, :, 0]
    g_comp = im_org[:, :, 1]
    b_comp = im_org[:, :, 2]
    if sign == 'blue':
        segm = (r_comp < 10) & (g_comp > 85) & (g_comp < 105) & \
                (b_comp > 180) & (b_comp < 200)
    if sign == 'red':
        segm = (r_comp > 160) & (r_comp < 180) & (g_comp > 50) & (g_comp < 80) & \
                    (b_comp > 50) & (b_comp < 80)
    return img_as_ubyte(segm)

In [34]:
blue_sign_img = detect_dtu_signs(im_org, 'blue')
io.imshow(blue_sign_img)
plt.title('Blue sign identified')
io.show()

## Exercise 14

In [36]:
red_sign_img = detect_dtu_signs(im_org, 'red')
io.imshow(red_sign_img, cmap = 'gray')
plt.title('Red sign identified')
io.show()

In [ ]:
r = 1530
c = 2250
print('Value of only R channel {}'.format(im_org[ r, c, 0]))
print('Value of only G channel {}'.format(im_org[ r, c, 1]))
print('Value of only B channel {}'.format(im_org[ r, c, 2]))

## Exercise 15

In [37]:
hsv_img = color.rgb2hsv(im_org)
hue_img = hsv_img[:, :, 0]
value_img = hsv_img[:, :, 2]
fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(8, 2))
ax0.imshow(im_org)
ax0.set_title("RGB image")
ax0.axis('off')
ax1.imshow(hue_img, cmap='hsv')
ax1.set_title("Hue channel")
ax1.axis('off')
ax2.imshow(value_img)
ax2.set_title("Value channel")
ax2.axis('off')

fig.tight_layout()
io.show()

In [40]:
r = 1800
c = 750
print('Value of only R channel {}'.format(hsv_img[ r, c, 0]))
print('Value of only G channel {}'.format(hsv_img[ r, c, 1]))
print('Value of only B channel {}'.format(hsv_img[ r, c, 2]))

In [45]:
segm_blue = (hue_img < 0.6) & (hue_img  > 0.55) & (value_img < 0.77) & (value_img > 0.70)
segm_red = (hue_img < 1.0) & (hue_img  > 0.99)

fig, (ax1, ax2) = plt.subplots(1,2,figsize = (10,5))
ax1.imshow(segm_blue, cmap = 'gray')
ax1.set_title('DTU sign image (Blue)')

ax2.imshow(segm_red, cmap = 'gray')
ax2.set_title('DTU sign image (Red)')
plt.show()

## Exercise 16

In [7]:
!python data/Ex3-VideoPixelWiseOperations.py

/bin/bash: /home/chris/anaconda3/envs/course02502/lib/python3.9/site-packages/cv2/../../../../lib/libtinfo.so.6: no version information available (required by /bin/bash)
Starting image capture
Opening connection to camera
[ WARN:0@0.009] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
Starting camera loop
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swra

In [12]:
!python data/exercise17.py

/bin/bash: /home/chris/anaconda3/envs/course02502/lib/python3.9/site-packages/cv2/../../../../lib/libtinfo.so.6: no version information available (required by /bin/bash)
Starting image capture
Opening connection to camera
[ WARN:0@0.008] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
Starting camera loop
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swra

In [26]:
!python data/exercise18.py

/bin/bash: /home/chris/anaconda3/envs/course02502/lib/python3.9/site-packages/cv2/../../../../lib/libtinfo.so.6: no version information available (required by /bin/bash)
Starting image capture
Opening connection to camera
[ WARN:0@0.016] global /croot/opencv-suite_1676452025216/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
Starting camera loop
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swra